In [8]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [9]:
from fastai import *
from fastai.vision.widgets import *
from fastbook import *

# Turning Your Model into an Online Application
We are now going to look at what it takes to turn this model into a working online application. We will just go as far as creating a basic working prototype.


## Using the Model for Inference
When we use a model for getting predictions, instead of training, we call it inference.
To create our inference learning from the exported file, we use *load_learner*.

In [10]:
path = Path()
learn_inf = load_learner(path/'export.pkl')

When we're doing inference, we're generally just getting predictions for one image at a time. To do this, pass a filename to *predict*.

In [11]:
learn_inf.predict('images/grizzly.jpg')

('grizzly', TensorBase(1), TensorBase([2.6949e-06, 9.9999e-01, 3.2075e-06]))

This has returned three things: the predicted category, the index of the predicted category, and the probability of each category.
The last two are based on the order of categories in the vocab of the *DataLoaders*.
At inference time, you can access the *DataLoaders* as an attribute of the *Learner*.

In [12]:
learn_inf.dls.vocab

['black', 'grizzly', 'teddy']

We now know how to make predictions from our saved model, so we have everything we need to start building our app.
We can do it directly in a Jupyter notebook.

# Creating a Notebook App from the Model

To use our model in an application, we can simply treat the *predict* method as a regular function. Therefore, creating an app from the model can be done using any of the myriad of frameworks and techniques available to application developers.

However, most data scientists are not familiar with web application development, so let's try creating a complete working web application using nothing but Jupyter notebooks!
The two things we need to make this happen are:
- IPython widgets (ipywidgets)
- Voila

IPython widgets are GUI components that bring together Javascript and Python functionality in a web browser, and can be created and used within a Jupyter notebook. However, we don't want to require users of our application to run Jupyter themselves.

That is why Voila exists. It is a system for making applications consisting of IPython widgets available to end users, without them having to use Jupyter at all. Viola is taking advantage of that fact that a notebook is *already* a kind of web application, just a rather complex one that depends on another web application: Jupyter itself. Essentially, it helps us automaticalyl conver the complex web application we've already implicity made (the notebook) into a simpler, easier-to-deploy web application, which functions like a normal web application rather than like a notebook.

But we still have the advantage of developing in a notebook, so with ipywidgets, we can build our GUI step by step. We will use this approach to create a simple image classifier.

In [13]:
# First, we need a file upload widget
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [15]:
# Now we can grab the image from the file upload
img = PILImage.create(btn_upload.data[-1])

In [16]:
# We can use an Output widget to display it
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [17]:
# Then we can get our prediction
pred, pred_idx, probs = learn_inf.predict(img)

In [18]:
# And use a Label to display them
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: grizzly; Probability: 1.0000')

In [19]:
# We'll need a button to do the classification. It looks exactly like the upload button.
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [ ]:
# We'll also need a click event handler, that is, a function that will be called when the button is clicked.
# We can just copy over the lines of code from above.
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

We can now put all our elements in a vertical box (VBox) to complete our GUI.

In [ ]:
# Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])

We have written all the code necessary for our app, The next step is to convert it into something we can deploy.